### Import Statements

In [89]:
import json
import os

### Create json File (need to change to json)

In [90]:
json_data = []

### Get Current Working Directory

In [91]:
cwd = "/data0/Data/composite_jsons"
stationary_video_path = "/home/timothyk/"
benchpress_cache = {}
squat_cache = {}
deadlift_cache = {}

### Checks if the video file is a stationary video

In [92]:
def is_stationary_video(file_name, file_folder):
    if file_folder == "benchpress" and benchpress_cache[file_name] == 2:
        return True
    elif file_folder == "squat" and squat_cache[file_name] == 2:
        return True
    elif file_folder == "deadlift" and deadlift_cache[file_name] == 2:
        return True
    return False

In [93]:
def stationary_videoset():
    input_files = [stationary_video_path + "benchpress_stationary.json",
                   stationary_video_path + "squat_stationary.json", 
                   stationary_video_path + "deadlift_stationary.json"]
    
    for i, path in enumerate(input_files):
        with open(path) as f:
            stationary_file = json.load(f)
        results = stationary_file['result']

        for dicts in results:
            file_name = dicts['filename']
            decision = dicts['decision']
            if i == 0:
                benchpress_cache[file_name] = decision
            if i == 1:
                squat_cache[file_name] = decision
            if i == 2:
                deadlift_cache[file_name] = decision

### Checks If Useful, And Add to JSON_DATA

In [94]:
def checkAdd(directory, filename, file_folder):
    # data is a json file
    path = directory + filename
    with open(path) as f:
        data = json.load(f)
    
    # A String
    macro = data["gt_labels"]["lift_type"]["macro"]
    # A json array 
    micro = data["gt_labels"]["lift_type"]["micro"]

    
    #added these###
    file_name = data["metadata"]["filename"] #this is in the format 303_333_n
    
    # Check conditions: microlabels don't contain accessory or none
    if is_stationary_video(file_name, file_folder) and "accessory" not in micro and "none" not in micro:
        jsonD = {}
        jsonD['composite_json'] = filename # this file has extention _liftype.json
        jsonD['macro'] = macro
        jsonD['micro'] = micro
        json_data.append(jsonD)


### Iterate Through Folders and Retreive Useful Files

In [95]:
stationary_videoset()
subdirs = [x[0] for x in os.walk(cwd)] 
for subdir in subdirs:  
    # pass cleanandjerk and snatch for now
    if subdir == cwd + "/cleanandjerk" or subdir == cwd + "/snatch":
        continue
    elif subdir == cwd + "/benchpress" or subdir == cwd + "/deadlift" or subdir == cwd + "/squat":
        file_folder = os.path.basename(subdir) #this is benchpress, squat, deadlift from composite_json
        for filename in os.listdir(subdir):
            if filename.endswith(".json"): 
                checkAdd(subdir + "/", filename, file_folder)
            else:
                continue

### Dump JSON File With Useful Info

In [96]:
with open("usable_micro_labeled_video_list.json", 'w+') as outfile:
    json.dump(json_data, outfile, indent=4, separators=(',', ': '))